In [8]:
!java -version


openjdk version "21.0.9" 2025-10-21
OpenJDK Runtime Environment (build 21.0.9+10-Ubuntu-124.04)
OpenJDK 64-Bit Server VM (build 21.0.9+10-Ubuntu-124.04, mixed mode, sharing)


In [9]:
import r5py

In [10]:
!pip uninstall -y r5py
!pip install r5py==0.2.0

Found existing installation: r5py 1.0.7
Uninstalling r5py-1.0.7:
  Successfully uninstalled r5py-1.0.7
ERROR: Could not find a version that satisfies the requirement r5py==0.2.0 (from versions: 0.0.3, 0.0.4, 0.1.0, 0.1.1.dev0, 0.1.1.dev2, 0.1.1, 0.1.2, 1.0.0.dev0, 1.0.0.dev1, 1.0.0.dev3, 1.0.0.dev9, 1.0.0.dev11, 1.0.0.dev12, 1.0.0, 1.0.1, 1.0.2, 1.0.3.dev0, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.1.0)
ERROR: No matching distribution found for r5py==0.2.0


In [11]:
!pip install r5py==1.0.7

  Using cached r5py-1.0.7-py3-none-any.whl.metadata (10.0 kB)
Using cached r5py-1.0.7-py3-none-any.whl (62 kB)


In [80]:
print(r5py.__version__)

1.0.7


In [5]:
import r5py
import geopandas
import geopandas as gpd
import pandas as pd
import datetime
import shapely

In [6]:
population_grid = geopandas.read_file("pop.gpkg")
Clinics = geopandas.read_file("Clinics.gpkg")

In [7]:
Clinics = Clinics.to_crs(population_grid.crs)

In [8]:
for c in population_grid.columns:
    print(f"'{c}'")

'population'
'id'
'geometry'


In [9]:
for c in Clinics.columns:
    print(f"'{c}'")

'DISTRICT'
'LONGITUDE'
'LATITUDE'
'UPDATED'
'NAMEOFFACI'
'OWNERSHIP'
'TYPEOFFACI'
'id'
'geometry'


In [10]:
overview_map = population_grid.explore("population", cmap="Purples")
overview_map = Clinics.explore(
    m=overview_map,
    marker_type="circle_marker",
    radius=6,
    color="red",
    fill=True,
    fill_color="red",
    fill_opacity=1,
    tooltip=["id"]
)

overview_map

In [12]:
network = r5py.TransportNetwork(
    osm_pbf="Zimbabwe.osm.pbf",
    gtfs=[],  # empty list
    elevation_model="Metro_Dem.tif"  # optional
)

In [13]:
# Prepare origins
origins = population_grid.copy()
# Project to metric CRS for accurate centroids
origins = origins.to_crs("EPSG:32736")
origins.geometry = origins.geometry.centroid
origins = origins.to_crs("EPSG:4326")  # back to geographic


In [14]:
# Prepare destinations
destinations = Clinics.copy()

In [15]:
#Compute the travel matrices 
travel_times = r5py.TravelTimeMatrix(
    network,
    origins=origins,
    destinations=destinations,
    departure=datetime.datetime.now(),
    transport_modes=[r5py.TransportMode.CAR],
    snap_to_network=True,
)

In [16]:
travel_times["to_id"].value_counts()

to_id
0     1432
1     1432
2     1432
3     1432
4     1432
5     1432
6     1432
7     1432
8     1432
9     1432
10    1432
11    1432
12    1432
13    1432
14    1432
15    1432
16    1432
17    1432
18    1432
19    1432
20    1432
21    1432
22    1432
23    1432
24    1432
25    1432
26    1432
27    1432
28    1432
29    1432
30    1432
31    1432
32    1432
33    1432
Name: count, dtype: int64

In [17]:
# Save to CSV
travel_times.to_csv("drive.csv", index=False)

In [18]:
# Quick peek at the first 10 rows
df = pd.read_csv("drive.csv")
df.head(10)

,from_id,to_id,travel_time
0,1,0,44
1,1,1,43
2,1,2,40
3,1,3,42
4,1,4,42
5,1,5,40
6,1,6,42
7,1,7,42
8,1,8,38
9,1,9,42


In [19]:
closest = (
    df.sort_values("travel_time")
      .groupby("from_id", as_index=False)
      .first()
)

In [20]:
closest.head(10)

,from_id,to_id,travel_time
0,1,26,33
1,2,26,27
2,3,26,25
3,4,26,25
4,5,26,24
5,6,29,28
6,7,26,25
7,8,26,24
8,9,26,24
9,10,29,17


In [21]:
# Merge the CAR travel times with your population grid
travel_times_gdf = population_grid.merge(
    closest,
    left_on="id",
    right_on="from_id"
)

In [23]:
import folium

# Create base map with clean basemap
overview_map = travel_times_gdf.explore(
    column="travel_time",
    cmap="Spectral_r",
    legend=True,
    tiles="CartoDB positron",
    tooltip=["id", "to_id", "travel_time", "population"]
)

#Add clinics layer
Clinics.explore(
    m=overview_map,
    marker_type="circle_marker",
    marker_kwds={
        "radius": 8,
        "fill": True,
        "fillOpacity": 1,
        "color": "black",
        "fill_color": "red"
    },
    tooltip=["id", "NAMEOFFACI"]
)

# Set initial map center (Bulawayo)
overview_map.location = [-20.15, 28.58]  # Bulawayo coordinates
overview_map.zoom_start = 11  # good metropolitan view

# Add title
title_html = """
<h3 align="center" style="font-size:20px">
<b>Driving Travel Time to Health Facilities – Bulawayo Metropolitan, Zimbabwe</b>
</h3>
"""
overview_map.get_root().html.add_child(folium.Element(title_html))

# Save interactive map
overview_map.save("bulawayo_travel_time_map.html")

overview_map